In [2]:
from efficient_kan import KAN

import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

from sklearn.model_selection import KFold
from sklearn.metrics import *

In [3]:
dataset_path = 'output_folder'

In [4]:
transform = transforms.Compose(
    [transforms.Resize(size = (220,220)),
     transforms.RandomRotation(degrees=20),
     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5], std=[0.5])
    ]
)

In [5]:
dataset = datasets.ImageFolder(dataset_path, transform = transform)

In [6]:
print("Informações sobre o Dataset: \n\n", dataset)
print("\nRótulos: ", dataset.class_to_idx)

Informações sobre o Dataset: 

 Dataset ImageFolder
    Number of datapoints: 24
    Root location: output_folder
    StandardTransform
Transform: Compose(
               Resize(size=(220, 220), interpolation=bilinear, max_size=None, antialias=True)
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1), scale=(0.9, 1.1), shear=[-10.0, 10.0])
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )

Rótulos:  {'Inliers': 0, 'Outliers': 1}


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = KAN([220*220*3, 100, 2])
model.to(device)

KAN(
  (layers): ModuleList(
    (0-1): 2 x KANLinear(
      (base_activation): ReLU()
    )
  )
)

In [8]:
num_epoch = 35
learning_rate = 0.001
k_folds = 5

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
kf = KFold(n_splits=k_folds, shuffle=True)

In [9]:
results = {}
train_array = []
test_array = []

for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f'Fold {fold+1}/{k_folds}')

    train_array.append(train_idx)
    test_array.append(test_idx)

    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    trainloader = DataLoader(train_subset, shuffle=True)
    testloader = DataLoader(test_subset, shuffle=False)

    for epoch in range(num_epoch):
        model.train()
        running_train_loss = 0.0

        for inputs, labels in trainloader:
            inputs = inputs.view(-1, 220*220*3).to(device)
            labels = labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

        train_loss = running_train_loss / len(trainloader)
        _, predicted = torch.max(outputs.data, 1)
        
        acc_treino = accuracy_score(labels.cpu().detach().numpy(), predicted.cpu().detach().numpy())

        print(f"Época {epoch + 1}/{num_epoch} - Perda no treinamento: {train_loss:.6f} - Acurácia: {acc_treino:.4f}")

    model.eval()

    with torch.no_grad():
        for images, labels in testloader:
            images = images.view(-1, 220*220*3).to(device)
            labels = labels.to(device)
        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
    
    acc_teste = accuracy_score(labels.cpu().detach().numpy(), predicted.cpu().detach().numpy())
    print(f'Acc para o Fold {fold+1}: {acc_teste:.4f}')
    results[fold] = acc_teste

    print("!!!Teste finalizado!!!")

print(f'\nResultados K-Fold: {results}')
print(f'Média Acc: {sum(results.values()) / k_folds}')

Fold 1/5
Época 1/35 - Perda no treinamento: 20.864795 - Acurácia: 1.0000
Época 2/35 - Perda no treinamento: 7.900224 - Acurácia: 1.0000
Época 3/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 4/35 - Perda no treinamento: 0.000001 - Acurácia: 1.0000
Época 5/35 - Perda no treinamento: 0.993954 - Acurácia: 1.0000
Época 6/35 - Perda no treinamento: 0.673373 - Acurácia: 1.0000
Época 7/35 - Perda no treinamento: 1.159344 - Acurácia: 1.0000
Época 8/35 - Perda no treinamento: 6.295254 - Acurácia: 0.0000
Época 9/35 - Perda no treinamento: 3.854708 - Acurácia: 1.0000
Época 10/35 - Perda no treinamento: 0.462033 - Acurácia: 1.0000
Época 11/35 - Perda no treinamento: 0.017225 - Acurácia: 1.0000
Época 12/35 - Perda no treinamento: 0.185145 - Acurácia: 1.0000
Época 13/35 - Perda no treinamento: 0.046675 - Acurácia: 1.0000
Época 14/35 - Perda no treinamento: 0.097370 - Acurácia: 1.0000
Época 15/35 - Perda no treinamento: 0.004628 - Acurácia: 1.0000
Época 16/35 - Perda no treinamento: 0.0

In [10]:
train_array

[array([ 0,  2,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 17, 18, 20, 21,
        22, 23]),
 array([ 0,  1,  2,  3,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21,
        22, 23]),
 array([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 12, 13, 14, 16, 17, 18, 19, 20,
        22, 23]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 15, 16, 17, 18, 19,
        21, 23]),
 array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18, 19,
        20, 21, 22])]

In [11]:
test_array

[array([ 1,  3, 12, 16, 19]),
 array([ 4,  5,  7, 17, 18]),
 array([ 6,  8, 11, 15, 21]),
 array([ 9, 10, 14, 20, 22]),
 array([ 0,  2, 13, 23])]